# Galton Data Analysis

## Standard Imports

In [2]:
# For creating dataframe
import pandas as pd
import numpy as np

#For visualizing null values
import missingno as msno

# For Data-Visualization
import matplotlib.pyplot as plt #for creating visualizations
import seaborn as sns
plt.rcParams["figure.figsize"] = "25,10"
plt.rcParams["legend.fontsize"] = 16
plt.rcParams["axes.labelsize"] = 16

# Importing Statsmodel
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import scipy

# Ignoring Warnings
import warnings 
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'missingno'

## Loading the Data

In [ ]:
data = pd.read_csv("galton-families.csv")

data.head()

In [ ]:
data.describe(include = "all").transpose()

In [ ]:
data.info()

In [ ]:
data.drop(["Unnamed: 0", "family", "childNum"], axis = 1, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
#Creating a mapper to change the column_type

typeMapper_default = {
    
    "gender" : "category"
}

data = data.astype(typeMapper_default)
data.reset_index(drop=True, inplace=True)

## Exploratory Data Analysis

### Nullity Analysis

In [ ]:
msno.matrix(data);

### Univariate Analysis

#### Real-valued Features

##### Father's Height

In [ ]:
sns.distplot(data["father"], color="red", bins=20);

##### Mother's Height

In [ ]:
sns.distplot(data["mother"], color="green", bins= 15)

##### MidParent height

In [ ]:
sns.distplot(data["midparentHeight"], color="black", bins= 15);

##### Child height

In [ ]:
sns.distplot(data["childHeight"], color="orange", bins= 15);

##### Number of Children

In [ ]:
sns.countplot(data["children"]);

#### Categorical Variables

##### Gender

In [ ]:
sns.countplot(data["gender"]);

### Bivariate Analysis

In [ ]:
g = sns.PairGrid(data, hue = "gender")
g = g.map_upper(plt.scatter)
g = g.map_lower(sns.kdeplot, shade = True, shade_lowest = False)
g = g.map_diag(plt.hist)
g = g.add_legend()

From the above pair plot, we can infer that both father and mother's height is positively correlated with the midParent height. Also, as the child's height increases, the chances of the child being a male is greater.

### Correlation Matrix

In [ ]:
sns.heatmap(data.corr(), cmap="PiYG", annot=True);

In [ ]:
mapping = {'male': 0, 'female': 1}
data = data.replace({'gender': mapping})
data.head()

## Regression Models

### Liner Regression

#### Using only Father as a predictor

In [ ]:
X = data["father"]
y = data["childHeight"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Note the difference in argument order
model = sm.OLS(y_train, X_train).fit()
predictions = model.predict(X_test) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
scipy.stats.pearsonr(y_test, predictions)

#### Using only Mother as a predictor

In [ ]:
X = data["mother"]
y = data["childHeight"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Note the difference in argument order
model = sm.OLS(y_train, X_train).fit()
predictions = model.predict(X_test) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
scipy.stats.pearsonr(y_test, predictions)

#### Using Father and Mother as a predictor

In [ ]:
X = data[["father", "mother"]]
y = data["childHeight"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Note the difference in argument order
model = sm.OLS(y_train, X_train).fit()
predictions = model.predict(X_test) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
scipy.stats.pearsonr(y_test, predictions)

#### Liner Regression using all the predictors

In [ ]:
X = data[["father",
          "mother",
          "midparentHeight",
          "children",
          "gender"]]

y = data["childHeight"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Note the difference in argument order
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X_test) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
scipy.stats.pearsonr(y_test, predictions)

### Standardizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_standard_scaled = pd.DataFrame(scaler.fit_transform(X), columns = data.columns.values.tolist()[1:6])

X_train, X_test, y_train, y_test = train_test_split(X_standard_scaled, y, random_state=42)

### Random Forest Regressor

In [ ]:
# Importing the random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import recall_score

# Evaluating model's area under curve ROC
from sklearn.metrics import r2_score, accuracy_score

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(X_train, y_train)

print(rf.score(X_test, y_test))

predictions = rf.predict(X_test)

In [ ]:
scipy.stats.pearsonr(y_test, predictions)

### Feature Important based on Random Forest Regressor

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_ ,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [ ]:
plt.bar(feature_importances.index, feature_importances.importance, color = "red");
plt.xlabel("Features");
plt.xticks(rotation=90);
plt.ylabel("Feature Importance");

### Deep Neural Network


In [ ]:
# Keras and TensorFlow imports for Deep Learning
import keras
import tensorflow as tf

# Using keras with a Scikit-learn wrapper
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

# for defining the Neural-net
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam

In [ ]:
def reg_model(learn_rate = 0.01):
    network = keras.models.Sequential()
    layer_1 = Dense(512, input_shape = (X_train.shape[1],), activation="relu")
    layer_2 = Dense(512, activation="tanh")
    layer_3 = Dense(512, activation="tanh")
    layer_4 = Dense(1)
    network.add(layer_1)
    network.add(Dropout(0.2))
    network.add(layer_2)
    network.add(Dropout(0.2))
    network.add(layer_3)
    network.add(Dropout(0.2))
    network.add(layer_4)
    network.compile(loss = "mean_squared_error",
                    optimizer = "adam",
                    metrics = ["mse"])
    return network

In [ ]:
epochs = 500
batch_size = 32
mlpModel = KerasRegressor(reg_model, epochs=epochs, batch_size=batch_size, verbose=1)
mlpModel.fit(X_train, y_train)
y_hat = mlpModel.predict(X_test)
rsquared  = r2_score(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
print(f"Model coefficient of determination, R^2={rsquared}")
print(f"MSE: {mse}")

In [ ]:
scipy.stats.pearsonr(y_test, y_hat)

### Kernel Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA, KernelPCA

kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
X_kpca = kpca.fit_transform(X_standard_scaled)
X_back = kpca.inverse_transform(X_kpca)
pca = PCA(n_components=5)

X_pca_5 = pca.fit_transform(X_back)

principal_X_pca_5 = pd.DataFrame(data = X_pca_5, columns=['principal component 1', 
                                                          'principal component 2',
                                                          'principal component 3', 
                                                          'principal component 4',
                                                          'principal component 5'])

### Linear Regression with 3 principal components as predictors


In [ ]:
X = principal_X_pca_5

y = data["childHeight"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_hat = model.predict(X_test)

r2_score(y_test, y_hat)

In [ ]:
scipy.stats.pearsonr(y_test, y_hat)

### Scree Plot

In [ ]:
explained_variance = np.var(X_pca_5, axis=0)
explained_variance_ratio = explained_variance / np.sum(explained_variance)

var1 = [np.cumsum(explained_variance_ratio)]
var1_new= var1[0].tolist()

N = [1,2,3,4,5]

PCA_table = pd.DataFrame(np.column_stack([N,var1_new]),columns = ['N', 'Cumulative_Variance'])

PCA_table

### Scree Plot

In [ ]:
plt.rcParams["figure.figsize"] = "20,15"
plt.plot(PCA_table.N, PCA_table.Cumulative_Variance , linewidth = 5, c = "red")

plt.xlabel("Number of Principal Components")
plt.xlim(0,5)
plt.ylabel("Cumulative Proportion of Variance Explained")

### K-PCA with 3 Principal Components 

In [ ]:
kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
X_kpca = kpca.fit_transform(X_standard_scaled)
X_back = kpca.inverse_transform(X_kpca)
pca = PCA(n_components=3)

X_pca_3 = pca.fit_transform(X_back)

In [ ]:
explained_variance = np.var(X_pca_3, axis=0)
explained_variance_ratio = explained_variance / np.sum(explained_variance)

explained_variance_ratio

np.cumsum(explained_variance_ratio)

In [ ]:
principal_X_pca_3 = pd.DataFrame(data=X_pca_3, columns=['principal component 1', 
                                                        'principal component 2',
                                                        'principal component 3'])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = "20,15"

fig = plt.figure()
axis = Axes3D(fig)

axis.scatter(principal_X_pca_3.iloc[:,0],
             principal_X_pca_3.iloc[:,1],
             principal_X_pca_3.iloc[:,2], c = data.gender, s = 100, alpha=0.6)

axis.set_xlabel("Principal Component 1")
axis.set_ylabel("Principal Component 2")
axis.set_zlabel("Principal Component 3")
plt.show